# Import packs

In [3]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Point Cloud Data Preparation

In [2]:
filefolder = r"D:/Downloads/ITC/"
dataname = "ITC_groundfloor.ply"

In [3]:
pcd = o3d.io.read_point_cloud(filefolder + dataname)

In [4]:
pcd_center = pcd.get_center()
pcd.translate(-pcd_center)

PointCloud with 334992 points.

In [5]:
o3d.visualization.draw_geometries([pcd])

# Point cloud Random Sampling

In [6]:
retained_ratio = 0.2
sampled_pcd = pcd.random_down_sample(retained_ratio)

In [7]:
o3d.visualization.draw_geometries([sampled_pcd], window_name = "Sampled Point Cloud")

# Stastical Outlier Removal

In [8]:
nn = 16
std_multiplier = 10
filtered_pcd,filtered_idx = pcd.remove_statistical_outlier(nb_neighbors=nn,std_ratio=std_multiplier)

In [9]:
outliers = pcd.select_by_index(filtered_idx,invert=True)
outliers.paint_uniform_color([1,0,0])
o3d.visualization.draw_geometries([filtered_pcd,outliers],window_name="Outlier Removal")


In [10]:
num = []
for i in range(10000,5):
    num.append(i)

In [11]:
str(num)

'[]'

# PointCloud Voxel Sampling

In [12]:
voxel_size = 0.05
pcd_downSampled  = filtered_pcd.voxel_down_sample(voxel_size=voxel_size)

In [13]:
o3d.visualization.draw_geometries([pcd_downSampled],window_name="Voxel Downsampled")

In [14]:
pcd_downSampled

PointCloud with 322996 points.

# PointCloud Normals Rxtraction

In [15]:
nn_distance = 0.05

In [16]:
radius_normals = nn_distance * 4
pcd_downSampled.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals,max_nn=30))

In [17]:
pcd_downSampled.paint_uniform_color([0.6,0.6,0.6])
o3d.visualization.draw_geometries([pcd_downSampled,outliers],window_name="Normal Estimation")

# RANSAC Parameter Setting

In [18]:
nn_distance = np.mean(pcd_downSampled.compute_nearest_neighbor_distance())

In [19]:
nn_distance

0.05188491177806045

# PointCloud Segmentation with RANSAC

In [20]:
distance_threshold = 0.1
ransac_n =3
num_iterations = 1000

In [21]:
plane_model, inliers = pcd.segment_plane(distance_threshold,ransac_n,num_iterations)
[a,b,c,d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

Plane equation: -0.00x + -0.00y + 1.00z + 1.35 = 0


In [22]:
inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)

#Paint the clouds
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud.paint_uniform_color([0.6, 0.6, 0.6])

radius_normals = nn_distance * 4
outlier_cloud.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals,max_nn=30))

#Visualize the inliers and outliers
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

# Scaling 3d Segmentation:Mi;to_order RANSAC

In [23]:
segment_models = {}
segments= {}

In [24]:
max_plane_idx = 10

In [25]:
rest=pcd
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(
    distance_threshold=0.1,ransac_n=3,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print("pass",i,"/",max_plane_idx,"done.")

pass 0 / 10 done.
pass 1 / 10 done.
pass 2 / 10 done.
pass 3 / 10 done.
pass 4 / 10 done.
pass 5 / 10 done.
pass 6 / 10 done.
pass 7 / 10 done.
pass 8 / 10 done.
pass 9 / 10 done.


In [26]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])

# DBSCAN for 3d point cloud Clustering

In [27]:
epsilon = 0.15
min_cluster_points = 5

In [28]:
rest=pcd
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(
    distance_threshold=0.1,ransac_n=3,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    labels = np.array(segments[i].cluster_dbscan(eps=epsilon, min_points=min_cluster_points))
    candidates=[len(np.where(labels==j)[0]) for j in np.unique(labels)]
    best_candidate=int(np.unique(labels)[np.where(candidates== np.max(candidates))[0]])
    rest = rest.select_by_index(inliers, invert=True) + segments[i].select_by_index(list(np.where(labels!=best_candidate)[0]))
    segments[i]=segments[i].select_by_index(list(np.where(labels== best_candidate)[0]))
    segments[i].paint_uniform_color(list(colors[:3]))
    print("pass",i,"/",max_plane_idx,"done.")

pass 0 / 10 done.
pass 1 / 10 done.
pass 2 / 10 done.
pass 3 / 10 done.
pass 4 / 10 done.
pass 5 / 10 done.
pass 6 / 10 done.
pass 7 / 10 done.
pass 8 / 10 done.
pass 9 / 10 done.


In [29]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])

There are several metrics that can be used to evaluate the quality of clustering results. Here are a few commonly used metrics:

1. **Silhouette score**: The silhouette score measures how similar a point is to its own cluster compared to other clusters. It ranges from -1 to 1, where a score of 1 indicates that the point is well-matched to its own cluster and poorly-matched to neighboring clusters, and a score of -1 indicates the opposite. A higher silhouette score indicates better clustering results.

2. **Calinski-Harabasz index**: The Calinski-Harabasz index measures the ratio of the between-cluster variance to the within-cluster variance. A higher index indicates better clustering results.

3. **Davies-Bouldin index**: The Davies-Bouldin index measures the average similarity between each cluster and its most similar cluster, normalized by the average dissimilarity between each cluster and its least similar cluster. A lower index indicates better clustering results.

4. **Visual inspection**: Sometimes, the best way to evaluate the quality of clustering results is to visually inspect the clusters and see if they make sense. This can be done using a 3D visualization library such as `matplotlib` or `mayavi`.

To calculate these metrics in Python, you can use the scikit-learn library. Here is an example code snippet:



In [30]:
o3d.visualization.draw_geometries([rest])

In [31]:
labels = np.array(rest.cluster_dbscan(eps=0.2, min_points=5))

In [32]:
max_label = labels.max()
colors = plt.get_cmap("tab20")(labels / (max_label 
if max_label > 0 else 1))
colors[labels < 0] = 0
rest.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([rest])

# Voxelization and Labeling

In [33]:
voxel_size = 0.2

In [34]:
min_bound = pcd.get_min_bound()
max_bound = pcd.get_max_bound()

In [35]:
pcd_ransac=o3d.geometry.PointCloud()
for i in segments:
 pcd_ransac += segments[i]

In [36]:
pcd_ransac.paint_uniform_color([1, 0, 0])

PointCloud with 262490 points.

In [37]:
voxel_grid_structural = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd_ransac, voxel_size=voxel_size)

In [38]:
rest.paint_uniform_color([0.1, 0.1, 0.8])
voxel_grid_clutter = o3d.geometry.VoxelGrid.create_from_point_cloud(rest, voxel_size=voxel_size)

In [39]:
o3d.visualization.draw_geometries([voxel_grid_clutter,voxel_grid_structural])

# Spatial Modelling

In [40]:
def fit_voxel_grid(point_cloud, voxel_size, min_b=False, max_b=False):
# This is where we write what we want our function to do.
    return voxel_grid, indices

Within the function, we will (1) Determine the minimum and maximum coordinates of the point cloud, (2) Calculate the dimensions of the voxel grid, (3) Create an empty voxel grid, (4) Calculate the indices of the occupied voxels and (5) Mark occupied voxels as True.

In [41]:
min_b = pcd.get_min_bound()
max_b = pcd.get_max_bound()

In [42]:
# Determine the minimum and maximum coordinates of the point cloud
if type(min_b) == bool or type(max_b) == bool:
  min_coords = np.min(point_cloud, axis=0)
  max_coords = np.max(point_cloud, axis=0)
else:
  min_coords = min_b
  max_coords = max_b
# Calculate the dimensions of the voxel grid
grid_dims = np.ceil((max_coords - min_coords) / voxel_size).astype(int)
# Create an empty voxel grid
voxel_grid = np.zeros(grid_dims, dtype=bool)
# Calculate the indices of the occupied voxels
indices = ((point_cloud - min_coords) / voxel_size).astype(int)
# Mark occupied voxels as True
voxel_grid[indices[:, 0], indices[:, 1], indices[:, 2]] = True

NameError: name 'point_cloud' is not defined

In [ ]:
voxel_size = 0.3

#get the bounds of the original point cloud
min_bound = pcd.get_min_bound()
max_bound = pcd.get_max_bound()

ransac_voxels, idx_ransac = fit_voxel_grid(pcd_ransac.points, voxel_size, min_bound, max_bound)
rest_voxels, idx_rest = fit_voxel_grid(rest.points, voxel_size, min_bound, max_bound)

#Gather the filled voxels from RANSAC Segmentation
filled_ransac = np.transpose(np.nonzero(ransac_voxels))

#Gather the filled remaining voxels (not belonging to any segments)
filled_rest = np.transpose(np.nonzero(rest_voxels))

#Compute and gather the remaining empty voxels
total = pcd_ransac + rest
total_voxels, idx_total = fit_voxel_grid(total.points, voxel_size, min_bound, max_bound)
empty_indices = np.transpose(np.nonzero(~total_voxels))

NameError: name 'indices' is not defined